In [5]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import os

In [6]:
data = pd.read_csv('data/train/trainx.txt', sep='\t')
features = data.columns.values
print("特征如下：{} \n".format(features))
print("数据数目: ", len(data))

特征如下：['ITEM_NAME' 'TYPE'] 

数据数目:  499447


## 数据清洗
* 替换一些字符 类似于 （ ） ‘ ’ _

In [7]:
import re
from zhon.hanzi import punctuation
from string import punctuation as english_punc
def  rm_punc(strs):
    return re.sub(r"[{}]+".format(punctuation + english_punc)," ",strs)
data[features[0]]  = data[features[0]].apply(rm_punc)
data.head()

,ITEM_NAME,TYPE
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值


In [8]:
# 因为有的label里本身就存在空格，所以先要把空格去掉，再去掉- - ， 再以空格划分
data = shuffle(data)
data_y = data[features[1]]

# 取训练数据和测试数据
train = data.sample(frac= 0.8).reset_index()
test = data.sample(frac= 0.2).reset_index()


print("训练数据数目:  ", len(train))
print("测试数据数目:  ", len(test))

train_x  = train[features[0]]
train_y  = train[features[1]]

test_x  = test[features[0]]
test_y  = test[features[1]]

print(train_x.shape, test_x.shape)

# 分出数据集 训练集 测试集 数据集用来训练词向量
data_y = data_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
data_y= [each.split() for each in data_y]

train_y = train_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
train_y = [each.split() for each in train_y]

test_y = test_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
test_y = [each.split() for each in test_y]

训练数据数目:   399558
测试数据数目:   99889
(399558,) (99889,)


## 分词

In [9]:
import jieba
from random import shuffle
def participle(data, doshuffle = False):
    words = []
    for i in range(len(data)):
        result=[]
        seg_list = jieba.cut(data.iloc[i])
        for w in seg_list :#读取每一行分词
            if w != " ":
                result.append(w)
        # 打乱每条数据的分词
        if doshuffle:
            shuffle(result)
        words.append(result)#将该行分词写入列表形式的总分词列表
    return words

In [10]:
# 每条数据的分词结果是个长度不等的词列表
word_data = participle(data[features[0]], doshuffle = False)
word_data_train = participle(train_x,doshuffle = False)
word_data_test = participle(test_x, doshuffle = False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Richado\AppData\Local\Temp\jieba.cache
Loading model cost 0.734 seconds.
Prefix dict has been built succesfully.


In [11]:
data[features[0]][0]

'腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费'

In [12]:
from random import shuffle
word_data[8]

['货到付款',
 '一步',
 '赢',
 '2014',
 '秋季',
 '松糕',
 '帆布鞋',
 '女',
 '韩版潮',
 '厚底',
 '松糕鞋',
 '学生',
 '高帮',
 '休闲',
 '板鞋',
 '黑',
 '白色',
 '35',
 '正规',
 '码']

## 标签分离(一个标签分成三个标签)

In [13]:
train_y = pd.DataFrame(train_y, columns=['label1', 'label2', 'label3'])
test_y = pd.DataFrame(test_y, columns=['label1', 'label2', 'label3'])
data_y = pd.DataFrame(data_y, columns=['label1', 'label2', 'label3'])

In [14]:
len(data_y["label1"].unique())

22

In [15]:
len(data_y["label2"].unique())

191

In [16]:
len(data_y["label3"].unique())

1192

## 标签onn_hot

In [17]:
from keras.utils import to_categorical
# 先生成唯一数组
y_label1 = data_y['label1'].unique().tolist()
y_label2 = data_y['label2'].unique().tolist()
y_label3 = data_y['label3'].unique().tolist()

# 获取在唯一数组中的索引(训练集和测试集各有3个标签需要处理)
train_y_label1_map = train_y['label1'].apply(lambda x: y_label1.index(x))
train_y_label2_map = train_y['label2'].apply(lambda x: y_label2.index(x))
train_y_label3_map = train_y['label3'].apply(lambda x: y_label3.index(x))
test_y_label1_map = test_y['label1'].apply(lambda x: y_label1.index(x))
test_y_label2_map = test_y['label2'].apply(lambda x: y_label2.index(x))
test_y_label3_map = test_y['label3'].apply(lambda x: y_label3.index(x))

Using TensorFlow backend.


In [18]:
# 生成对应one-hot(用做训练模型的标签)
train_y_label1_one_hot = to_categorical(train_y_label1_map, 22)
train_y_label2_one_hot = to_categorical(train_y_label2_map, 191)
train_y_label3_one_hot = to_categorical(train_y_label3_map, 1192)
test_y_label1_one_hot = to_categorical(test_y_label1_map, 22)
test_y_label2_one_hot = to_categorical(test_y_label2_map, 191)
test_y_label3_one_hot = to_categorical(test_y_label3_map, 1192)

# 词向量训练

In [19]:
from gensim.models.word2vec import Word2Vec 
if os.path.exists("w2v_model"):
    model = Word2Vec.load("w2v_model")
    print(1)
else:
    sentences = word_data
    model= Word2Vec(size=50, window=10, min_count = 1)
    model.build_vocab(sentences)
    model.train(sentences,total_examples = model.corpus_count,epochs = model.iter)
    model.save("w2v_model")

1


## 填充词向量(构造输入输出)（让每个句子拥有同样的维度）

In [20]:
from keras.preprocessing import sequence
x_train = [model[word] for word in word_data_train]
x_test = [model[word] for word in word_data_test]
x_train = sequence.pad_sequences(x_train, maxlen=18)
x_test  = sequence.pad_sequences(x_test, maxlen=18) 

c:\users\richado\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
c:\users\richado\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


## 默认参数


## TextCnn

In [27]:
import tensorflow as tf
import numpy as np


class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, maxlen, sequence_length, num_classes, filter_sizes, embedding_size, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.float32, [None, sequence_length, embedding_size], name="input_x")
        self.input_y1 = tf.placeholder(tf.float32, [None, num_classes[0]], name="input_y1")
        self.input_y2 = tf.placeholder(tf.float32, [None, num_classes[1]], name="input_y2")
        self.input_y3 = tf.placeholder(tf.float32, [None, num_classes[2]], name="input_y3")
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        keep_prob = 0.9
        pooled_outputs = []
        self.input_x_expand = tf.expand_dims(self.input_x, -1)
        print(self.input_x_expand.shape)
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                # 卷积
                conv = tf.nn.conv2d(
                    self.input_x_expand,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
        
        
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            
                # 最大池化
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled = tf.nn.dropout(pooled, keep_prob)
                pooled_outputs.append(pooled)
                

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        print(self.h_pool.shape)
        
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        self.h_drop = self.h_pool_flat
        
        # Final (unnormalized) scores and predictions
        
        with tf.name_scope("output1"):
            W1 = tf.get_variable(
                "W11",
                shape=[num_filters_total, 64],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[64]), name="b11")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output1")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, keep_prob)
            W2 = tf.get_variable(
                "W12",
                shape=[64, num_classes[0]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[0]]), name="b12")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores1 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions1 = tf.argmax(self.scores1, 1, name="predictions")
            
        with tf.name_scope("output2"):
            W1 = tf.get_variable(
                "W21",
                shape=[num_filters_total, 128],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[128]), name="b21")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output2")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, keep_prob)
            W2 = tf.get_variable(
                "W22",
                shape=[128, num_classes[1]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[1]]), name="b22")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores2 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions2 = tf.argmax(self.scores2, 1, name="predictions")
            
        with tf.name_scope("output3"):
            W1 = tf.get_variable(
                "W31",
                shape=[num_filters_total, 256],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[256]), name="b31")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output3")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, keep_prob)
            W2 = tf.get_variable(
                "W32",
                shape=[256, num_classes[2]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[2]]), name="b32")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores3 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions3 = tf.argmax(self.scores3, 1, name="predictions")
        # Calculate mean cross-entropy loss
        
        with tf.name_scope("loss"):
            losses1 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores1, labels=self.input_y1)
            losses2 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores2, labels=self.input_y2)
            losses3 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores3, labels=self.input_y3)
            self.loss = 0.1 * tf.reduce_mean(losses1) + 0.2 * tf.reduce_mean(losses2) + 0.7*tf.reduce_mean(losses3) + l2_reg_lambda * l2_loss
            tf.summary.scalar('loss1',tf.reduce_mean(losses1))
            tf.summary.scalar('loss2',tf.reduce_mean(losses2))
            tf.summary.scalar('loss3',tf.reduce_mean(losses3))
            tf.summary.scalar('loss',self.loss)
        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions1 = tf.equal(self.predictions1, tf.argmax(self.input_y1, 1))
            correct_predictions2 = tf.equal(self.predictions2, tf.argmax(self.input_y2, 1))
            correct_predictions3 = tf.equal(self.predictions3, tf.argmax(self.input_y3, 1))
#           单独的准确率
            self.accuracy1 = tf.reduce_mean(tf.cast(correct_predictions1, "float"), name="accuracy1")
            self.accuracy2 = tf.reduce_mean(tf.cast(correct_predictions2, "float"), name="accuracy2")
            self.accuracy3 = tf.reduce_mean(tf.cast(correct_predictions3, "float"), name="accuracy3")
#           一起的准确率
            self.acc = tf.reduce_mean(tf.cast(correct_predictions1, "float")*tf.cast(correct_predictions2, "float")*tf.cast(correct_predictions3, "float"))
            tf.summary.scalar('acc1',self.accuracy1)
            tf.summary.scalar('acc2',self.accuracy2)
            tf.summary.scalar('acc3',self.accuracy3)
            tf.summary.scalar('acc',self.acc)

In [28]:
import math
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
  
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = np.min(((batch_num + 1) * batch_size, data_size))
            yield shuffled_data[start_index:end_index]
            

In [31]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn

# 特征数这里是第三级标签的特征数
max_features = 1192

# 句子填充的长度
maxlen = 16

batch_size = 256 

epochs = 100

is_training = True
# 词向量长度
embedding_dims = 50

filters = 64
kernel_size = 5
hidden_dims = 128
epochs = 100

# Parameters
# ==================================================

# Data loading params
# tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
# tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
# tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
# tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
# tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
# tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
# tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
# tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
# tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
# tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
# tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
# tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
# tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS

def train(x_train, y_train, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
#           allow_soft_placement=FLAGS.allow_soft_placement,
#           log_device_placement=FLAGS.log_device_placement
        )
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(maxlen = 10, sequence_length = 18, num_classes = [22, 191, 1192], filter_sizes = [1,2, 3, 4, 5], embedding_size = 50, num_filters = 64)

            # Define Training procedure

            global_step = tf.Variable(0, name="global_step", trainable=False)
#           学习率衰减
            learning_rate = tf.train.exponential_decay(
                0.01,
                global_step,
                x_train.shape[0] / batch_size,
                0.99,
                staircase=True,)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())
            
            merged = tf.summary.merge_all()
            
            writer = tf.summary.FileWriter("fwwb",sess.graph)
            acc_sum = np.array([0.0,0.0,0.0,0.0])
            loss_sum = 0
            
            def train_step(x_batch, y_batch1, y_batch2, y_batch3, loss_sum, acc_sum, interal = 10):
  
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y1: y_batch1,
                  cnn.input_y2: y_batch2,
                  cnn.input_y3: y_batch3
                }
#         三个标签单独的准确率， 一起的准确率共四个准确率
                rs, _, step,  loss, accuracy1, accuracy2, accuracy3, acc = sess.run(
                    [merged, train_op, global_step, cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                acc_sum += [accuracy1, accuracy2, accuracy3, acc]
                loss_sum += loss
                if step % interal == 0:
                    print("{}: step {}, loss {:g}, acc {}".format(time_str, step, loss_sum/interal, acc_sum/interal))
                    loss_sum = 0
                    acc_sum = np.array([0.0,0.0,0.0,0.0])
                    writer.add_summary(rs, step)
                return loss_sum, acc_sum
            # 评估步骤
            def dev_step(x_batch, y_batch):
                """
                Evaluates model on a dev set
                """

                dev_batches = batch_iter(
                    list(zip(x_batch, y_batch[0], y_batch[1], y_batch[2])), 640, 1)
                step = 0
                loss = 0
                accuracy = np.array([0.0, 0.0, 0.0,0.0])
                for batch in dev_batches:
                    x_batch_dev, y_batch1_dev, y_batch2_dev, y_batch3_dev = zip(*batch)
                    feed_dict = {
                      cnn.input_x: x_batch_dev,
                      cnn.input_y1: y_batch1_dev,
                      cnn.input_y2: y_batch2_dev,
                      cnn.input_y3: y_batch3_dev
                    }
                    step,  temp_loss, temp_accuracy1, temp_accuracy2, temp_accuracy3 , acc= sess.run(
                        [global_step,cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                        feed_dict)
                    accuracy[0] += temp_accuracy1 * len(x_batch_dev)
                    accuracy[1] += temp_accuracy2 * len(x_batch_dev)
                    accuracy[2] += temp_accuracy3 * len(x_batch_dev)
                    accuracy[3] += acc * len(x_batch_dev)
                    loss += temp_loss * len(x_batch_dev)
                accuracy /= x_batch.shape[0]
                loss /= x_batch.shape[0]
                time_str = datetime.datetime.now().isoformat()                    

                print("{}: step {}, loss {:g}, acc {} ".format(time_str, step, loss, accuracy))


            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train[0],y_train[1], y_train[2] )), batch_size, 200)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch1, y_batch2, y_batch3 = zip(*batch)
                loss_sum, acc_sum = train_step(x_batch, y_batch1, y_batch2, y_batch3,  loss_sum, acc_sum, 40)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 200 == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev)
                    print("")

In [ ]:
y_train = [train_y_label1_one_hot,train_y_label2_one_hot,train_y_label3_one_hot]
y_test = [test_y_label1_one_hot,test_y_label2_one_hot,test_y_label3_one_hot]
train(x_train, y_train,  x_test, y_test)



(?, 18, 50, 1)
(?, 1, 1, 320)
2019-02-22T00:00:48.701899: step 40, loss 2.81225, acc [0.73701172 0.57236328 0.45078125 0.38056641]
2019-02-22T00:00:50.010345: step 80, loss 1.75888, acc [0.82949219 0.71572266 0.60439453 0.53583984]
2019-02-22T00:00:51.284568: step 120, loss 1.58793, acc [0.83134766 0.72978516 0.61855469 0.55146484]
2019-02-22T00:00:52.542918: step 160, loss 1.46455, acc [0.8359375  0.74003906 0.6421875  0.57509766]
2019-02-22T00:00:53.807220: step 200, loss 1.38436, acc [0.846875   0.75048828 0.65107422 0.58085937]

Evaluation:
2019-02-22T00:00:58.070829: step 200, loss 1.35978, acc [0.84823154 0.74875112 0.65562775 0.58517955] 

2019-02-22T00:00:59.430365: step 240, loss 1.33695, acc [0.85214844 0.76015625 0.66240234 0.59619141]
2019-02-22T00:01:00.733355: step 280, loss 1.31144, acc [0.85517578 0.75849609 0.66269531 0.59501953]
2019-02-22T00:01:02.018489: step 320, loss 1.27594, acc [0.84853516 0.76416016 0.66933594 0.59785156]
2019-02-22T00:01:03.300649: step 360, l

2019-02-22T00:03:02.168972: step 2640, loss 0.972568, acc [0.87910156 0.80556641 0.72763672 0.66816406]
2019-02-22T00:03:03.352924: step 2680, loss 0.995203, acc [0.87802734 0.80253906 0.728125   0.66337891]
2019-02-22T00:03:04.530425: step 2720, loss 0.984882, acc [0.87880859 0.80634766 0.73076172 0.66689453]
2019-02-22T00:03:05.701976: step 2760, loss 0.972547, acc [0.88242188 0.80537109 0.73007813 0.66660156]
2019-02-22T00:03:06.876997: step 2800, loss 0.973306, acc [0.88222656 0.80947266 0.73242188 0.66992188]

Evaluation:
2019-02-22T00:03:10.817713: step 2800, loss 0.924814, acc [0.87973651 0.80943848 0.74170329 0.67841304] 

2019-02-22T00:03:12.188159: step 2840, loss 1.00711, acc [0.87529297 0.79785156 0.72304687 0.66015625]
2019-02-22T00:03:13.356732: step 2880, loss 0.983258, acc [0.87734375 0.80390625 0.728125   0.66347656]
2019-02-22T00:03:14.533739: step 2920, loss 0.9785, acc [0.88164062 0.80087891 0.73417969 0.66865234]
2019-02-22T00:03:15.705289: step 2960, loss 0.973722

2019-02-22T00:05:13.524466: step 5240, loss 0.881746, acc [0.88535156 0.81308594 0.75244141 0.68583984]
2019-02-22T00:05:14.701968: step 5280, loss 0.877933, acc [0.88388672 0.81298828 0.74853516 0.68105469]
2019-02-22T00:05:15.929566: step 5320, loss 0.87886, acc [0.88457031 0.81435547 0.75566406 0.68505859]
2019-02-22T00:05:17.098140: step 5360, loss 0.868386, acc [0.89111328 0.81992188 0.75527344 0.68925781]
2019-02-22T00:05:18.260267: step 5400, loss 0.880052, acc [0.88652344 0.81738281 0.75185547 0.68769531]

Evaluation:
2019-02-22T00:05:22.149431: step 5400, loss 0.859703, acc [0.88581325 0.81609587 0.75650972 0.68893472] 

2019-02-22T00:05:23.387410: step 5440, loss 0.892311, acc [0.88085938 0.81396484 0.74892578 0.6828125 ]
2019-02-22T00:05:24.545570: step 5480, loss 0.89723, acc [0.87792969 0.81210938 0.746875   0.68320313]
2019-02-22T00:05:25.770689: step 5520, loss 0.885205, acc [0.88691406 0.81386719 0.75224609 0.68789062]
2019-02-22T00:05:26.944719: step 5560, loss 0.89342

2019-02-22T00:07:21.302813: step 7800, loss 0.814288, acc [0.88762527 0.82127161 0.7673918  0.69849533] 

2019-02-22T00:07:22.755097: step 7840, loss 0.804678, acc [0.88698015 0.82155244 0.77107106 0.70030086]
2019-02-22T00:07:23.913256: step 7880, loss 0.776009, acc [0.88916016 0.82431641 0.77666016 0.70078125]
2019-02-22T00:07:25.072406: step 7920, loss 0.806369, acc [0.88730469 0.81855469 0.76865234 0.69667969]
2019-02-22T00:07:26.227093: step 7960, loss 0.801337, acc [0.89433594 0.82011719 0.76904297 0.69941406]
2019-02-22T00:07:27.392690: step 8000, loss 0.773807, acc [0.89306641 0.82402344 0.77529297 0.70429688]

Evaluation:
2019-02-22T00:07:31.273931: step 8000, loss 0.81201, acc [0.88786553 0.82430498 0.76961427 0.7013485 ] 

2019-02-22T00:07:32.521820: step 8040, loss 0.777576, acc [0.89160156 0.82783203 0.77138672 0.70126953]
2019-02-22T00:07:33.690890: step 8080, loss 0.797261, acc [0.89042969 0.8234375  0.76796875 0.69560547]
2019-02-22T00:07:34.866407: step 8120, loss 0.80

2019-02-22T00:09:27.263010: step 10400, loss 0.782049, acc [0.88964844 0.82460937 0.77861328 0.70566406]

Evaluation:
2019-02-22T00:09:31.272712: step 10400, loss 0.783966, acc [0.88958744 0.82525603 0.77608145 0.70271   ] 

2019-02-22T00:09:32.654025: step 10440, loss 0.801961, acc [0.89199219 0.82382813 0.76914063 0.69931641]
2019-02-22T00:09:33.821112: step 10480, loss 0.818616, acc [0.88359375 0.82138672 0.76464844 0.69492188]
2019-02-22T00:09:34.990181: step 10520, loss 0.81765, acc [0.89267578 0.82373047 0.76367188 0.69257813]
2019-02-22T00:09:36.220756: step 10560, loss 0.817349, acc [0.88603516 0.81503906 0.76816406 0.69404297]
2019-02-22T00:09:37.386851: step 10600, loss 0.831481, acc [0.88427734 0.81923828 0.76259766 0.69511719]

Evaluation:
2019-02-22T00:09:41.289868: step 10600, loss 0.780058, acc [0.89039834 0.82685781 0.77550081 0.70563325] 

2019-02-22T00:09:42.545243: step 10640, loss 0.813036, acc [0.88955078 0.82070312 0.76601562 0.6953125 ]
2019-02-22T00:09:43.708857

...

2019-02-22T00:56:11.657694: step 66840, loss 0.608954, acc [0.89667969 0.8453125  0.81982422 0.73212891]
2019-02-22T00:56:12.821806: step 66880, loss 0.593953, acc [0.89433594 0.84189453 0.81962891 0.73320312]
2019-02-22T00:56:13.989388: step 66920, loss 0.585719, acc [0.90078125 0.84433594 0.82236328 0.734375  ]
2019-02-22T00:56:15.153993: step 66960, loss 0.592295, acc [0.896875   0.84775391 0.81972656 0.73417969]
2019-02-22T00:56:16.316614: step 67000, loss 0.608729, acc [0.89267578 0.84140625 0.81298828 0.72382813]

Evaluation:
2019-02-22T00:56:20.184481: step 67000, loss 0.656521, acc [0.89782659 0.84093344 0.81610588 0.72930953] 

2019-02-22T00:56:21.426401: step 67040, loss 0.602237, acc [0.89658203 0.83857422 0.81689453 0.72392578]
2019-02-22T00:56:22.590015: step 67080, loss 0.611411, acc [0.89423828 0.83798828 0.81494141 0.72587891]
2019-02-22T00:56:23.753629: step 67120, loss 0.592743, acc [0.89814453 0.84335938 0.82167969 0.73535156]
2019-02-22T00:56:25.053194: step 67160, 

2019-02-22T00:58:20.108370: step 69400, loss 0.657297, acc [0.89762636 0.84142398 0.81791789 0.73149195] 

2019-02-22T00:58:21.527423: step 69440, loss 0.589829, acc [0.89589844 0.8390625  0.821875   0.72958984]
2019-02-22T00:58:22.833390: step 69480, loss 0.577486, acc [0.90078125 0.84736328 0.82431641 0.73515625]
2019-02-22T00:58:24.068470: step 69520, loss 0.554125, acc [0.89853516 0.84638672 0.82978516 0.7375    ]
2019-02-22T00:58:25.251387: step 69560, loss 0.587547, acc [0.89755859 0.84433594 0.825      0.73408203]
2019-02-22T00:58:26.435833: step 69600, loss 0.586865, acc [0.90058594 0.84619141 0.82158203 0.73535156]

Evaluation:
2019-02-22T00:58:30.633391: step 69600, loss 0.649833, acc [0.89753626 0.84163421 0.81813813 0.73021053] 

2019-02-22T00:58:31.963660: step 69640, loss 0.59135, acc [0.8921875  0.84130859 0.82216797 0.72773438]
2019-02-22T00:58:33.187787: step 69680, loss 0.586603, acc [0.89638672 0.84101563 0.82070312 0.73134766]
2019-02-22T00:58:34.428325: step 69720,

2019-02-22T01:00:31.634967: step 72000, loss 0.542314, acc [0.89892578 0.84101563 0.83964844 0.74072266]

Evaluation:
2019-02-22T01:00:35.723488: step 72000, loss 0.655839, acc [0.89743616 0.84150407 0.81753747 0.72898918] 

2019-02-22T01:00:37.016558: step 72040, loss 0.54552, acc [0.89980469 0.84550781 0.83847656 0.74296875]
2019-02-22T01:00:38.208445: step 72080, loss 0.544976, acc [0.90029297 0.84951172 0.83496094 0.74111328]
2019-02-22T01:00:39.413726: step 72120, loss 0.563617, acc [0.90048828 0.84355469 0.83027344 0.73662109]
2019-02-22T01:00:40.615529: step 72160, loss 0.576816, acc [0.89482422 0.84765625 0.82929688 0.73310547]
2019-02-22T01:00:41.942823: step 72200, loss 0.552757, acc [0.90126953 0.8453125  0.83076172 0.73828125]

Evaluation:
2019-02-22T01:00:46.084417: step 72200, loss 0.658024, acc [0.89773649 0.84188449 0.81984002 0.732433  ] 

2019-02-22T01:00:47.347729: step 72240, loss 0.574003, acc [0.89716797 0.83994141 0.83232422 0.73417969]
2019-02-22T01:00:48.533662

2019-02-22T01:02:47.522405: step 74560, loss 0.584291, acc [0.89677734 0.84150391 0.82138672 0.73056641]
2019-02-22T01:02:48.823411: step 74600, loss 0.582727, acc [0.89746094 0.84169922 0.82470703 0.73671875]

Evaluation:
2019-02-22T01:02:52.926368: step 74600, loss 0.647579, acc [0.89760634 0.84322598 0.81944959 0.73204257] 

2019-02-22T01:02:54.329499: step 74640, loss 0.570566, acc [0.90039062 0.84853516 0.82363281 0.73544922]
2019-02-22T01:02:55.643401: step 74680, loss 0.583454, acc [0.90068359 0.84033203 0.8234375  0.73232422]
2019-02-22T01:02:56.936968: step 74720, loss 0.589424, acc [0.89365234 0.83896484 0.81748047 0.72666016]
2019-02-22T01:02:58.233547: step 74760, loss 0.558118, acc [0.89941406 0.84921875 0.82880859 0.73857422]
2019-02-22T01:02:59.537493: step 74800, loss 0.570713, acc [0.9015625  0.84394531 0.82226562 0.73222656]

Evaluation:
2019-02-22T01:03:03.412278: step 74800, loss 0.65075, acc [0.89880768 0.84280551 0.81992011 0.73272332] 

2019-02-22T01:03:04.739036

2019-02-22T01:05:05.041725: step 77120, loss 0.565721, acc [0.89931641 0.84267578 0.82597656 0.73105469]
2019-02-22T01:05:06.266347: step 77160, loss 0.559454, acc [0.89892578 0.84326172 0.82890625 0.73242188]
2019-02-22T01:05:07.518746: step 77200, loss 0.57194, acc [0.8953125  0.84140625 0.82851562 0.73193359]

Evaluation:
2019-02-22T01:05:11.750116: step 77200, loss 0.650347, acc [0.8983772  0.84285557 0.82112144 0.7323429 ] 



In [ ]:
test_y_label3_one_hot.shape

In [ ]:
train_y_label3_one_hot.shape